In [1]:
import nltk
import string
import re
import pytesseract
import cv2
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from pdf2image import convert_from_path 
from textblob import TextBlob

To scan the pdf and recognize the text in it

In [2]:
def scan(PDF_file,id=""):
    pages = convert_from_path(PDF_file, 500, poppler_path="C:\\Program Files\\poppler-21.11.0\\Library\\bin")
    image_counter = 1
    for page in pages:
        filename = id+"_page_"+str(image_counter)+".jpg"
        page.save(filename, 'JPEG')
        image_counter = image_counter + 1

    pytesseract.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract.exe"
    out = id+"_ans.txt"
    f=open(out,"a")
    for i in range(1, image_counter):
        filename = id+"_page_"+str(i)+".jpg"
        img=cv2.imread(filename)
        grayscale = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        adaptive =cv2.adaptiveThreshold(grayscale,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,31,9)
        text = str(pytesseract.image_to_string(adaptive))
        text = text.replace('-\n', '')
        out+=text
        f.write(text)
    f.close()

To break the answer into topics

In [3]:
def breakList(answer):
    shead=re.findall(r'[\n]*[a-zA-Z]+[\s]*[-]*[a-zA-Z]*:',answer)
    heads=[]
    for x in shead:
        x=x.lstrip("\n")
        heads+=[str(x).replace(':','')]
    return answer,heads

To Lemmattize or stem the words and normalizing the compounds

In [4]:
def lemmatized(words):
    lemmatizer=WordNetLemmatizer()
    words=[lemmatizer.lemmatize(word) for word in words]
    return words

To filter the dictionary of paragraphs to the required simple words

In [5]:
def filter(para):
    para,heads=breakList(para)
    words=word_tokenize(para)
    headWords=[word.lower() for word in heads.copy()]
    words=[word.lower() for word in words.copy()]
    words=[str(TextBlob(word).correct()) for word in words.copy()]
    headWords=[str(TextBlob(word).correct()) for word in headWords.copy()]
    for word in words.copy():
        if word in string.punctuation:
            words.remove(word)
    words=[word for word in words.copy() if word not in stopwords.words() and word.isalnum()]
    words=lemmatized(words)
    headWords=lemmatized(headWords)
    return [words,headWords]

To get the percentage of matching in between two lists

In [6]:
def match(student, scheme):
    k,n,matchPercent=0,0,0
    for x in range(2):
        for word in scheme[x]:
            n+=1
            if word in student[x]:
                k+=1
    matchPercent=(k/n)
    return matchPercent

To calculate the score based on the weights assigned and correcting the error percentage of the OCR model

In [7]:
def calcScore(compulsoryWordsMatch,keyWordsMatch,compulsoryWordsWeight,maxAllotedScore):
    score=((compulsoryWordsMatch*compulsoryWordsWeight)+(keyWordsMatch*(100-compulsoryWordsWeight)))*maxAllotedScore/100
    return score

Function to validate an answer

In [8]:
def validateQuestion(studentAnswer="", schemeAnswer="", compulsoryWords="", compulsoryWordsWeight=80, maxAllotedScore=10):
    keyWords=filter(studentAnswer)
    schemeKeyWords=filter(schemeAnswer)
    compulsoryWords=filter(compulsoryWords)
    
    compulsoryWordsMatch=match(keyWords,compulsoryWords)
    keyWordsMatch=match(keyWords,schemeKeyWords)
    
    score=calcScore(compulsoryWordsMatch,keyWordsMatch,compulsoryWordsWeight,maxAllotedScore)
    return score

To post the score into the original students dictionary

In [9]:
def postScore(studentId,score):
    students[studentId].scores[ans]=score

To Preprocess the text file to classify the answers one by one

In [10]:
def preprocess(file):
    s=open(file,'r+').read()
    ansIds=re.findall(r'[0-9]+\)',s)
    ans=re.split(r'[0-9]+\)',s)
    ans=[x for x in ans.copy() if x!='']
    answers=dict()
    for n,a in zip(ansIds,ans):
        answers[int(n.split(')')[0])]=a
    return answers,len(answers)

To represent data of a student such as his/her answers(dictionary), scores(dictionary) awarded for each question and total score

In [11]:
class student:
    def __init__(self,answers,scores,totalScore=0):
        self.answers=answers
        self.scores=scores
        self.totalScore=totalScore

Taking all the details as input

In [12]:
#To store the data of students with ID as key and student object as value
students=dict()
n=int(input("Enter No. of students: "))
for _ in range(n):
    id=input("Enter student ID: ")
    scan(f"{id}.pdf",id)
    f=id+"_ans.txt"
    answers,count=preprocess(f)
    scores={}
    for i in range(count):
        scores[i+1]=0
    students[id]=student(answers,scores)

#To store the scheme data for each question with Question number as key
scan("Scheme.pdf","Scheme")
scheme,x=preprocess("Scheme_ans.txt")

#To store the compulsory words in answer for each question with Question number as key
scan("Compulsory.pdf","Compulsory")
compulsoryWords,x=preprocess("Compulsory_ans.txt")

#To store the maximum alloted scores for each questions with Question number as key
maxAllotedScores=dict()
for i in range(count):
        maxAllotedScores[i+1]=10

To validate each and every question of each and every student and posting their scores

In [13]:
for s in students.keys():
    for ans in students[s].answers.keys():
        score=validateQuestion(students[s].answers[ans],scheme[ans],compulsoryWords[ans],80,maxAllotedScores[ans])
        postScore(s,score)
    students[s].totalScore=sum([sc for sc in students[s].scores.values()])

To print the final scores of each question and total score of each student

In [14]:
for s in students.keys():
    print(s,end="    ")
    for ans in students[s].answers.keys():
        print(ans,":","{:.2f}".format(students[s].scores[ans]),end="    ")
    print("{:.2f}".format(students[s].totalScore))

19131A0531    2 : 9.14    1 : 9.59    18.73
19131A0562    1 : 8.50    2 : 9.01    17.51
